# Calculation of Bader charges
The approach uses additional tool developled in [Henkelman group](http://theory.cm.utexas.edu/henkelman/code/bader/).

Download [binary 'bader'](http://theory.cm.utexas.edu/henkelman/code/bader/download/bader_lnx_64.tar.gz) for Bader analysis and put it to *~/tools/* folder on your cluster.

Additionally, download [binary 'vtstscripts'](http://theory.cm.utexas.edu/code/vtstscripts.tgz) and uncompress it to *~/tools/* folder as vts/ on your cluster.

In [1]:
#Usual preamble
from siman import header
from siman.header import db
from siman.SSHTools import SSHTools
from siman.calc_manage import add_loop, res_loop
from siman.set_functions import read_vasp_sets
from siman.database import read_database, write_database
from siman.analysis import calc_oxidation_states

read_database()
header.PATH2PROJECT    = 'topologic' # path to project relative to your home folder on cluster
header.PATH2POTENTIALS = '/home/aksenov/scientific_projects/PAW_PBE_VASP' #path to VASP POTENTIALS
header.varset['static'].potdir = {83:'Bi_pv', 34:'Se'} #subfolders with required potentials
header.ssh_object = SSHTools()
header.ssh_object.setup(user="aksenov",host="10.30.16.62",pkey="/home/aksenov/.ssh/id_rsa")

/home/aksenov/simanrc.py was read


In [5]:
#Creating new set 'bader' with parameters for Bader charges calculation
bader_pack = {'PREC':'Accurate', 'ADDGRID':'.TRUE.', 'EDIFF':1e-08, 
              'LAECHG':'.TRUE.', 'NELM':100, 'NSW':0, 'ICHARG':1, 'savefile' : 'acox'}
read_vasp_sets([('bader', 'static', bader_pack, 'override')]) #new set 'bader' from 'static'


Attention! You have chosen to override set bader
 

Warning! You did not change  NSW  in bader set
 



{'dos': <set_functions.InputSet at 0x7f1e6225d8d0>,
 'band': <set_functions.InputSet at 0x7f1e254b40f0>,
 'static': <set_functions.InputSet at 0x7f1e254aafd0>,
 'opt': <set_functions.InputSet at 0x7f1e254a2eb8>,
 'bader': <set_functions.InputSet at 0x7f1e254a2da0>}

In [7]:
add_loop('Bi2Se3', 'bader', 1, input_geo_file = 'Bi2Se3_mp-541837_computed.POSCAR', run = 1) # Submitting VASP job

-- Attention!, cluster None is not found, using default cee 

-- check_kpoints(): Kpoint   mesh is:  [8, 8, 8] 

-- check_kpoints(): The actual k-spacings are  [ 0.22  0.22  0.22] 

-- POSCAR was written to Bi2Se3//Bi2Se3.bader/1.POSCAR 

-- Attention! ngkpt =  [8, 8, 8]  is adopted from struct_des which you provided for it  Bi2Se3  and kspacing =  0.235 


Calculation ('Bi2Se3', 'bader', 1) successfully created

 

JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            169829       AMG KVPO516.  aksenov  R 2-23:23:45      1 node-amg01
            168188       AMG LTP_sa2_ asergeev  R 6-14:04:15      1 node-amg04
Submitted batch job 180604
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            180604       AMG Bi2Se3.b  aksenov PD       0:00      1 (None)
            169829       AMG KVPO516.  aksenov  R 2-23:23:45      1 node-amg01
            168188       AMG LTP_sa2_ asergeev  R 6-14:04:15      1 node-amg04 

-- 

'Bi2Se3'

In [10]:
res_loop('Bi2Se3', 'bader', 1) #reading results after the job is finished
write_database() # writing database



Max. F. tot  (meV/A) = 
[59]; 

-- name                          |  energy(eV)|    Vector lenghts (A)   | Stresses (MPa)     | N MD, N SCF    
-- Bi2Se3.bader.1                |  -20.1524  |10.27;10.27;10.27|  -684,-684,-685    |   1,18, 18    

Database has been successfully updated



In [2]:
header.warnings = 'yY'
# Calculate Bader charges on cluster 
db['Bi2Se3', 'bader', 1].get_bader_ACF()




Max. F. tot  (meV/A) = 
[59]; 

-- name                          |  energy(eV)|    Vector lenghts (A)   | Stresses (MPa)     | N MD, N SCF    
-- Bi2Se3.bader.1                |  -20.1524  |10.27;10.27;10.27|  -684,-684,-685    |   1,18, 18    
-- /home/aksenov/topologic/Bi2Se3//Bi2Se3.bader/1.CHGCAR_sum does not exist. trying to calculate it  

Atoms in file1: 5, Atoms in file2: 5
Points in file1: 2744000, Points in file2: 2744000
 

-- Warning! File  /home/aksenov/topologic/Bi2Se3//Bi2Se3.bader/1.CHGCAR does not exist. Checking .gz ..  


 

ACF_text =  X           Y           Z       CHARGE      MIN DIST   ATOMIC VOL
 --------------------------------------------------------------------------------
    1    2.416678    1.435730   11.588585   20.170149     0.923287    27.967000
    2    3.648748    2.167694   17.496674   20.170148     0.923287    27.966944
    3    0.000000    0.000000    0.000000    6.694249     0.990643    29.794916
    4    1.308403    0.777313    6.274127    6.4

[20.17, 20.17, 6.694, 6.466, 6.468]

In [3]:
print(db['Bi2Se3', 'bader', 1].charges) # print calculated Bader charges

[20.17, 20.17, 6.694, 6.466, 6.468]


In [4]:
#Calculate oxidation states


calc_oxidation_states( db['Bi2Se3', 'bader', 1] )


Bi 0.8
Bi 0.8
Se -0.7
Se -0.5
Se -0.5
